In [1]:
#!/usr/bin/env python
# Simple trader script to orchestrate price monitoring and order placement

import os
import sys
import time
import logging
import subprocess
import signal
import argparse
from datetime import datetime


# Use current working directory instead of __file__
log_dir = os.path.join(os.getcwd(), 'output', 'logs')
os.makedirs(log_dir, exist_ok=True)

# # Set up logging
# log_dir = os.path.join(os.path.dirname(__file__), 'output', 'logs')
# os.makedirs(log_dir, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir, "trader.log")),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Flag to track if we should continue running
running = True

def signal_handler(sig, frame):
    """Handle Ctrl+C and other termination signals"""
    global running
    logger.info("Received termination signal. Shutting down gracefully...")
    running = False

def run_price_monitor():
    """Run the price monitor script"""
    logger.info("Starting price monitor...")
    
    # Path to your existing price_monitor.py
    script_path = os.path.join(os.getcwd(), 'price_monitor.py')
    
    try:
        # Run price_monitor with a fixed runtime
        cmd = [sys.executable, script_path, '--runtime', '60']
        process = subprocess.Popen(cmd)
        
        # Wait for process to complete
        process.wait()
        
        if process.returncode == 0:
            logger.info("Price monitor completed successfully")
            return True
        else:
            logger.error(f"Price monitor exited with code {process.returncode}")
            return False
    
    except Exception as e:
        logger.error(f"Error running price monitor: {str(e)}")
        return False

def run_order_placement():
    """Run the order placement script"""
    logger.info("Starting order placement...")
    
    # Path to your existing test_order_2.py script (or the modified version)
    script_path = os.path.join(os.getcwd(), 'vertical_spread_order.py')
    
    try:
        # Generate a random client ID to avoid connection conflicts
        from random import randint
        client_id = randint(100, 9999)
        
        # Run the order placement script
        cmd = [sys.executable, script_path, '--client', str(client_id)]
        process = subprocess.Popen(cmd)
        
        # Wait for process to complete
        process.wait()
        
        if process.returncode == 0:
            logger.info("Order placement completed successfully")
            return True
        else:
            logger.error(f"Order placement exited with code {process.returncode}")
            return False
    
    except Exception as e:
        logger.error(f"Error running order placement: {str(e)}")
        return False

def main():
    """Main function to run the trading system"""
    # Register signal handlers for graceful shutdown
    signal.signal(signal.SIGINT, signal_handler)
    signal.signal(signal.SIGTERM, signal_handler)
    
    logger.info("=" * 80)
    logger.info("Starting automated trading system")
    logger.info("=" * 80)
    
    cycle_count = 0
    
    # Main loop
    while running:
        cycle_count += 1
        logger.info(f"Starting cycle {cycle_count}")
        
        # Step 1: Run price monitor
        monitor_success = run_price_monitor()
        
        # Step 2: Run order placement if monitoring was successful
        if monitor_success and running:
            order_success = run_order_placement()
        
        # Wait for 1 minute before starting the next cycle
        if running:
            logger.info("Waiting 60 seconds until next cycle...")
            wait_start = time.time()
            while running and (time.time() - wait_start < 60):
                time.sleep(1)
    
    logger.info("Trading system shutdown complete")
    return 0

if __name__ == "__main__":
    sys.exit(main())

2025-04-11 05:37:13,944 - INFO - ================================================================================
2025-04-11 05:37:13,946 - INFO - Starting automated trading system
2025-04-11 05:37:13,946 - INFO - ================================================================================
2025-04-11 05:37:13,947 - INFO - Starting cycle 1
2025-04-11 05:37:13,948 - INFO - Starting price monitor...
/home/chris_s_dodd/source/optcom/monitor/price_monitor.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2025-04-11 05:37:15,185 - INFO - Connecting to IBKR at 127.0.0.1:7497
2025-04-11 05:37:15,247 

SystemExit: 0

/home/chris_s_dodd/anaconda3/envs/optcom/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
